In [ ]:
import api
import datasets
import pandas as pd

from models import CommandConfiguration

In [ ]:
base = "http://192.168.67.3:7172"
# base = "http://127.0.0.1:7172"

In [ ]:
print(f"is_connected: {api.check_connection(base)}")

In [ ]:
data = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all", split="validation")
df = pd.DataFrame(data)
level1 = [d["task_id"] for d in data if d["Level"] == "1" and d["file_path"] != ""]

In [ ]:
command = CommandConfiguration(
    auto_run=True,
    os_mode=False,
    model="",
    api_base="",
    api_key="",
    system_prompt="""You are a general AI assistant.
I will ask you a question. Report your thoughts, and finish your answer with the following template: FINAL ANSWER: [YOUR FINAL ANSWER].
YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings.
If you are asked for a number, don’t use comma to write your number neither use units such as $ or percent sign unless specified otherwise.
If you are asked for a string, don’t use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise.
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string."""
)

In [ ]:
# api.run_single(base, command, level1[0])
for task_id in level1:
    try:
        result = api.run_single(base, command, task_id, timeout_s=30 * 60)
        print(result)
    except Exception as e:
        print(e)
        raise e